In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *
from fastai import sgdr

In [3]:
PATH = '/home/ubuntu/projects/dantegen/'

In [4]:
#fpath = get_file(PATH + 'data/raw/divcomm.txt', origin="http://www.gutenberg.org/files/1012/1012-0.txt")
fpath = PATH + 'data/raw/divcomm.txt'
text = open(fpath,encoding='utf8').read().lower()
text = text[932:-19658]

In [5]:
print(text[:18])

la divina commedia


In [6]:
chars = sorted(list(set(text)))
chars.insert(0, "\0")
vocab_size = len(chars)
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}
idx = [char_indices[c] for c in text]

In [7]:
maxlen = 40
c_in_dat = [[idx[i+j] for i in range(maxlen)] for j in range(len(idx)-maxlen-1)]
xs = np.stack(c_in_dat, axis=0)
c_out_dat = [idx[j+maxlen] for j in range(len(idx)-maxlen-1)]
y = np.stack(c_out_dat)

In [8]:
"".join(indices_char[i] for i in xs[0])

'la divina commedia\n  di dante alighieri\n'

In [9]:
xs.shape,y.shape

((561053, 40), (561053,))

In [10]:
n_hidden = 256
n_fac = 42

In [ ]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]))#, dim=-1)

In [11]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

NameError: name 'CharRnn' is not defined

In [12]:
val_idx = get_cv_idxs(len(idx)-maxlen-1)
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [ ]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [ ]:
t = m.e(V(torch.stack(xs)))
t.size()

In [ ]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

In [ ]:
t = m(*V(xs)); t.size()

In [ ]:
fit(m, md, 4, opt, F.nll_loss)

In [ ]:
set_lrs(opt, 1e-4)

In [ ]:
fit(m, md, 2, opt, F.nll_loss)

In [ ]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [ ]:
def get_next_n(inp = "nel mezzo del cammin di nostra vita\n  mi", n = 500):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [ ]:
print(get_next_n())

In [ ]:
len(text)*0.2

In [ ]:
PATH='/home/ubuntu/projects/dantegen/data/processed/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

In [ ]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=40; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

In [ ]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [147]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(torch.stack(cs)), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp)[-1])#.view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [ ]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [14]:
# new version
m = CharSeqStatefulLSTM(vocab_size, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [ ]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [ ]:
t = m.e(V(torch.stack(xs)))
t.size()

In [ ]:
#ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t,m.h)
#outp.size(), hn.size()

In [ ]:
F.log_softmax(m.l_out(outp[-1]))

In [15]:
fit(m, md, 2, lo.opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.75168  1.70017]                                
[ 1.       1.66213  1.60879]                                



In [124]:
on_end = lambda sched, cycle: save_model(m, f'/home/ubuntu/projects/dantegen/models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

A Jupyter Widget

[ 0.       1.5162   1.49679]                                
[ 1.       1.55491  1.49257]                                
[ 2.       1.45726  1.4401 ]                                
[ 3.       1.54527  1.49621]                                
[ 4.       1.49515  1.44461]                                
[ 5.       1.42361  1.39992]                                
[ 6.       1.39102  1.38575]                                
[ 7.       1.52513  1.46943]                                
 12%|█▏        | 105/877 [00:11<01:22,  9.30it/s, loss=1.49]

KeyboardInterrupt: 

In [ ]:
on_end = lambda sched, cycle: save_model(m, f'/home/ubuntu/projects/dantegen/models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

In [121]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    p = to_np(p)[0]
    ps= np.exp(p)/np.sum(np.exp(p))
    c = np.random.choice(chars,p=ps)
    return c

In [122]:
def get_next_n(inp = "nel mezzo del cammin di nostra vita\n  mi", n = 500):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [125]:
print(get_next_n())

nel mezzo del cammin di nostra vita
  mi già la conven: le lube apretta,
  prïete al cielo, m’eranda,
  di vota bratin d’altro mio erpe’
  cangue ciò fratian più luce m’appreso?
  e p’è ne la fosse d’era destro,
  lisura rima come s’aggià non poco».

  «se non è ’l vofo mar dolce amore;
  però ancula e l’un dì lo rimordivo
  più ligo a molt’ esser donce la trove,
  me l’amor del duca piro e ’l ci f’onde

  tu s’a lui, ch’un morti il mio più sopra
  bun erando di presto come pirmi».

  la vostra ben fosse, unenata, e le sappende,
  qua


In [126]:
maxlen = 125
c_in_dat = [[idx[i+j] for i in range(maxlen)] for j in range(len(idx)-maxlen-1)]
xs = np.stack(c_in_dat, axis=0)
c_out_dat = [idx[j+maxlen] for j in range(len(idx)-maxlen-1)]
y = np.stack(c_out_dat)

In [127]:
"".join(indices_char[i] for i in xs[0])

'la divina commedia\n  di dante alighieri\n\n\n\n\n\n  inferno\n\n\n\n\n  inferno • canto i\n\n\n  nel mezzo del cammin di nostra vita\n  mi r'

In [128]:
xs.shape,y.shape

((560968, 125), (560968,))

In [129]:
n_hidden = 256
n_fac = 42

In [130]:
val_idx = get_cv_idxs(len(idx)-maxlen-1)
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [131]:
m = CharSeqStatefulLSTM(vocab_size, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [132]:
fit(m, md, 2, lo.opt, F.nll_loss)

A Jupyter Widget

  3%|▎         | 29/877 [00:09<04:47,  2.95it/s, loss=3.08]

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/ubuntu/src/anaconda3/envs/dantegen/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/src/anaconda3/envs/dantegen/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/src/anaconda3/envs/dantegen/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



[ 0.       1.74525  1.68769]                                
[ 1.       1.65003  1.59278]                                



In [141]:
def get_next_n(inp = "nel mezzo del cammin di nostra vita\n  mi ritrovai per una selva oscura,\n  ché la diritta via era smarrita.\n\n  ahi quanto a di", n = 500):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [142]:
print(get_next_n())

nel mezzo del cammin di nostra vita
  mi ritrovai per una selva oscura,
  ché la diritta via era smarrita.

  ahi quanto a dispenti unò sì caggia;

  io dretti linanti
  che trova ad ogna vive a qual non facea.

  ché tre ond’ escettei è segna e fortar travilse.

  vide a la fioglio d’intenia veni;
  qual gente che più li fuor dire,
  nallo e quelli;
  e al passo è che intelli!e quinal
  da spezio fui fui si sovrar che sótte».

  quelli maltro parriste; tua come sempore
  ustri •’rima che s’i’ sù del dietto;
  e non fra cavenza rinaffene
  tra quest’ io den vorso al son pastì
  si fchenza, natura di non fuor reggiume»


In [143]:
on_end = lambda sched, cycle: save_model(m, f'/home/ubuntu/projects/dantegen/models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**3-1, lo.opt, F.nll_loss, callbacks=cb)

A Jupyter Widget

[ 0.       1.5276   1.48436]                                
[ 1.       1.53805  1.49429]                                
[ 2.       1.47523  1.43446]                                
[ 3.       1.54139  1.49952]                                
  9%|▉         | 81/877 [00:26<04:19,  3.07it/s, loss=1.53]

KeyboardInterrupt: 

In [144]:
print(get_next_n())

nel mezzo del cammin di nostra vita
  mi ritrovai per una selva oscura,
  ché la diritta via era smarrita.

  ahi quanto a dio s’acchie seggio giù dier fui
  ciò intro ben, non rispresso, sì che farendo.

  e i’ oltre a noi fosti di man canno:
  madito si pentian consonte guido.

  né semplerando membra la terra e votto,
  nel guir consonerlo di mente innogue!».

  poi che sì, che s’ele per la scende da frata,
  e con s’intende ne’ fuzgo te, la parte.

  e ’l poi dicendo: «per le mente, poi’,
  se sue me, ch’ei facear lo splosta rege,

  ch’ello lumo per speradicata sira.

  ché ’ sempre pozzo, lei, e segutare,
  lamb


In [145]:
print(get_next_n())

nel mezzo del cammin di nostra vita
  mi ritrovai per una selva oscura,
  ché la diritta via era smarrita.

  ahi quanto a dio torna le de la fin sappe,
  che ’l santo diss’ attentemploser mondo.

  o è assà ’l bommo non vidi dal nami
  d’insiù le infiuta, e poscian tolte.


  ma sì così cose al bel amera».

  «grazia farghio la mia nome provi
  s’ecchi l’un ne l’ostra predano in quella
  gente per trabbrado del penna savi,
  sì chi sapvia, i vista vedeva e forte;
  discese al nestavato anche: «tanto neste,
  e disorrare come sì vemiglio serta,
  sì che tu si regito, mostri vito aggiute;
  e la bocca spirti de le bene


In [146]:
n_hidden = 512
n_fac = 42

In [148]:
m = CharSeqStatefulLSTM(vocab_size, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [149]:
fit(m, md, 2, lo.opt, F.nll_loss)

A Jupyter Widget

  1%|          | 7/877 [00:05<11:19,  1.28it/s, loss=3.46]

Exception in thread Thread-18:
Traceback (most recent call last):
  File "/home/ubuntu/src/anaconda3/envs/dantegen/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/src/anaconda3/envs/dantegen/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/src/anaconda3/envs/dantegen/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



[ 0.       2.26346  2.21009]                                
[ 1.       1.89582  1.84159]                                



In [150]:
print(get_next_n())

nel mezzo del cammin di nostra vita
  mi ritrovai per una selva oscura,
  ché la diritta via era smarrita.

  ahi quanto a di la, diobperdo né dissa

  che ’l si vlente
  prentima per viute ditia per sersio viei di biùllurò du la e pabpe che avonta esa ribore’ la la ricar senzansi la prinaso,
  quammigna;
  le qualtun prellorar vite inciro mi non ’tro alfonio e dio s’acchie fassi ledere ossor col non tredia
  «io comperduladieio
  lerò arba,
  che peri,  il vivò la non consondenama pr’vigar che negla car onfioi ché se vera in più fio bei che arsìi imbesce
  sambronda:
  fïanmo disce inflalper preso,
  suvo quidase fos


In [151]:
save_model(m,'/home/ubuntu/projects/dantegen/models/512cells125ch2ep')